In [1]:
import os
import glob
import pandas as pd

In [2]:
pwd

'/Users/michaelpiacentino/Drive/git/2021_nSMase2_Ceramide_Endocytosis_EMT/Figure4/Laminin'

In [3]:
# Add analysis date here to apply to dataframe
analysis_date = '20210903'

path = os.path.abspath('')+'/raw_data_csvs/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Split values Image name columns
    (df['ExptDate'], df['Treatment'], df['Stains'], df['Embryo'], df['Somites'], df['ImageMag'], 
        df['Section']) = zip(*df['Image'].map(lambda x: x.split('_')))
    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,Date,Experiment,Image,Target,Treatment,Dose,Embryo,Somites,Section,Control Side,Experimental Side,ExptDate,Stains,ImageMag
0,20210824,20210824_pCISMPD3FLAG_BRE_TCFLef,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;BREH2B...,SMPD3-FLAG,SMPD3FLAG,3 µg/µl,BF;Pax7;H2BRFP;BREH2Bd2EGFP;Laminin,Emb2,sec1.czi,Complete Channel,No Channel,20210824,3ugul,8ss
1,20210824,20210824_pCISMPD3FLAG_BRE_TCFLef,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;BREH2B...,SMPD3-FLAG,SMPD3FLAG,3 µg/µl,BF;Pax7;H2BRFP;BREH2Bd2EGFP;Laminin,Emb3,sec1.czi,Complete Channel,Incomplete Channel,20210824,3ugul,9ss
2,20210824,20210824_pCISMPD3FLAG_BRE_TCFLef,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;BREH2B...,SMPD3-FLAG,SMPD3FLAG,3 µg/µl,BF;Pax7;H2BRFP;BREH2Bd2EGFP;Laminin,Emb3,sec2.czi,Complete Channel,Complete Channel,20210824,3ugul,9ss
3,20210824,20210824_pCISMPD3FLAG_BRE_TCFLef,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;BREH2B...,SMPD3-FLAG,SMPD3FLAG,3 µg/µl,BF;Pax7;H2BRFP;BREH2Bd2EGFP;Laminin,Emb3,sec3.czi,Complete Channel,Complete Channel,20210824,3ugul,9ss
4,20210824,20210824_pCISMPD3FLAG_BRE_TCFLef,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;BREH2B...,SMPD3-FLAG,SMPD3FLAG,3 µg/µl,BF;Pax7;H2BRFP;BREH2Bd2EGFP;Laminin,Emb4,sec1.czi,Complete Channel,Complete Channel,20210824,3ugul,9ss


In [4]:
# Get a list of treatments
treatment_list = full_df.Treatment.unique()
treatment_list = treatment_list.tolist()

# Group dataframe by treatment
df_byTreatment = (full_df.groupby(['Treatment', 'Image'])
                  ['Image', 'Treatment', 'Control Side', 'Experimental Side']).sum()

# Loop trough treatments, performing each analysis and exporting CSV file for each treatment
for i in treatment_list:
    treatment = i
    df_treatment = pd.DataFrame(df_byTreatment.xs(treatment))

    cntl_scores = pd.DataFrame(df_treatment['Control Side'].value_counts())
    expt_scores = pd.DataFrame(df_treatment['Experimental Side'].value_counts())
    results = pd.concat([cntl_scores, expt_scores], axis=1, sort=True)
    results.fillna(0, inplace=True)
    results = results.astype(int)
    results.to_csv(analysis_date + '_' + treatment + '_LamininResults.csv')